# Nested Queries and Set Operations


In [4]:
import warnings
warnings.filterwarnings('ignore')
from sols7 import *

In [5]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [6]:
%sql postgresql://aljamo01:@localhost/employees

'Connected: aljamo01@employees'

In [49]:
%%sql
SELECT *
FROM employees
LIMIT(1)

1 rows affected.


emp_no,birth_date,first_name,last_name,gender,hire_date
10001,1953-09-02,Georgi,Facello,M,1986-06-26


In [50]:
%%sql
SELECT *
FROM departments
LIMIT(1)

1 rows affected.


dept_no,dept_name
d001,Marketing


In [51]:
%%sql
SELECT *
FROM titles
LIMIT(1)

1 rows affected.


emp_no,title,from_date,to_date
10001,Senior Engineer,1986-06-26,9999-01-01


### 1.  How many employees have never worked for the quality management department?

In [10]:
%%sql 
SELECT count(emp_no)
FROM ((SELECT emp_no FROM employees) EXCEPT (SELECT emp_no FROM departments natural join dept_emp WHERE dept_name = 'Quality Management')) a natural join employees

1 rows affected.


count
279907


In [11]:
test_subq1(_)

Correct!


### 1.a  Select the first and last names of the first 10 employees ordered by last name, first name that have never worked for the development department.

In [13]:
%%sql
SELECT first_name, last_name
FROM ((SELECT emp_no FROM employees) EXCEPT (SELECT emp_no FROM departments natural join dept_emp WHERE dept_name = 'Development')) a natural join employees
ORDER BY last_name, first_name
LIMIT (10)

10 rows affected.


first_name,last_name
Aemilian,Aamodt
Aleksander,Aamodt
Alexius,Aamodt
Alois,Aamodt
Amabile,Aamodt
Anestis,Aamodt
Anwar,Aamodt
Arlette,Aamodt
Arumugam,Aamodt
Arunachalam,Aamodt


In [14]:
test_subq1a(_)

Correct!


### 2.  Give the first name, last name of the first 10 employees that have not been a Senior Engineer or Senior Staff. Order by employee number. "First" means by assigned employees number and not by hire date.

In [62]:
%%sql
SELECT distinct first_name, last_name
FROM employees natural join (SELECT emp_no, title FROM titles
WHERE title != 'Senior Engineer' and title != 'Senior Staff' ORDER BY emp_no LIMIT (10)) a

9 rows affected.


first_name,last_name
Bezalel,Simmel
Tzvetan,Zielinski
Duangkaew,Piveteau
Saniya,Kalloufi
Mary,Sluis
Sumant,Peac
Patricio,Bridgland
Chirstian,Koblick
Kyoichi,Maliniak


In [63]:
test_subq2(_)

Error! Results are not the same length. I was expecting 10 rows and your result has 9 rows.


AssertionError: 

### 3.  How many departments has each manager managed?

Your result should be in alphabetical order by last name, then first name

In [23]:
%%sql
SELECT first_name, last_name, count(dept_name)
FROM employees natural join (SELECT emp_no, dept_name FROM departments NATURAL JOIN dept_manager) a
GROUP BY emp_no
ORDER BY last_name, first_name


24 rows affected.


first_name,last_name,count
Ebru,Alpin,1
Tonny,Butterworth,1
Rosine,Cools,1
Leon,DasSarma,1
Oscar,Ghazalie,1
Marjo,Giarratana,1
DeForest,Hagimont,1
Rutger,Hofmeyr,1
Przemyslawa,Kaelbling,1
Hilary,Kambil,1


In [24]:
test_subq3(_)

Correct!


### 4.  How many people were ever managed by Tonny Butterworth? Consider all the cases.

In [66]:
SELECT (*)
FROM (
        SELECT from_date, to_date, dept_no
        FROM dept_manager Natural Join employees
        WHERE first_name= 'Tonny' and last_name = 'Butterworth') A

(psycopg2.ProgrammingError) unterminated quoted string at or near "'1988-10-17"
LINE 3: ...vice' and to_date>= '1985-01-01' and from_date <= '1988-10-1...
                                                             ^
 [SQL: "SELECT count(*) from dept_emp\nnatural join departments\nwhere dept_name = 'Customer Service' and to_date>= '1985-01-01' and from_date <= '1988-10-17"]


In [68]:
%%sql
SELECT count(*) from dept_emp
natural join departments
where dept_name = 'Customer Service' and to_date>= '1985-01-01' and from_date <= '1988-10-17

(psycopg2.ProgrammingError) unterminated quoted string at or near "'1988-10-17"
LINE 3: ...vice' and to_date>= '1985-01-01' and from_date <= '1988-10-1...
                                                             ^
 [SQL: "SELECT count(*) from dept_emp\nnatural join departments\nwhere dept_name = 'Customer Service' and to_date>= '1985-01-01' and from_date <= '1988-10-17"]


In [67]:
test_subq4(_)

Error! You have a different number of columns than expected.


TypeError: unsupported operand type(s) for +: 'int' and 'str'

### 5.  How many people have not ever been a manager?

In [38]:
%%sql
SELECT COUNT(*) 
FROM ((SELECT emp_no FROM employees) EXCEPT SELECT emp_no FROM titles WHERE title= 'Manager') a

1 rows affected.


count
300000


In [39]:
test_subq5(_)

Correct!


### 6.  What is the average salary for the 1980's?  What is the average salary for the 1990's?

Lets define this as salary level that start and end in the same decade.  We will leave out any salaries that cross the decade boundary.

Format the salaries using '$999,999.99'.

Your final answer should have two columns and two rows.  The columns should be labeled decade and average.

In [40]:
%%sql
SELECT '1980' decade, to_char(avg(salary), '$ 999,999.99') average FROM salaries WHERE '1980-01-01' <= from_date and to_date <= '1989-12-31'
UNION
SELECT '1990' decade, to_char(avg(salary), '$ 999,999.99') average FROM salaries WHERE '1990-01-01' <= from_date and to_date <= '1999-12-31'

2 rows affected.


decade,average
1980,"$ 54,985.51"
1990,"$ 62,281.51"


In [41]:
test_subq6(_)

Error! Line by line comparison shows that your query is invalid. Read the question carefully.
Your solution should be like the following table:
  decade       average
0   1980  $  54,985.51
1   1990  $  62,281.51



AssertionError: 

### 7.  How many people have had 5 or more raises since May 5, 1979?

In [42]:
%%sql
SELECT COUNT(*)
FROM (SELECT emp_no, count(*) FROM salaries WHERE to_date > '1979-05-05'
     GROUP BY emp_no) a WHERE count >= 5

1 rows affected.


count
245432


In [43]:
test_subq7(_)

Correct!
